In [23]:
import pandas as pd
from pathlib import Path

repo = Path('..').resolve()
omxspi_path = '../omxspi_members.csv'
master_path = '../instrument_master.csv'
coverage_path = '../coverage_table_updated.csv'

# Read OMXSPI members (no header in the file)
omxspi = pd.read_csv(omxspi_path, sep='\t', header=None, 
                     names=['Fullname', 'CCY', 'Last', 'Change', 'ChangePct', 
                            'Bid', 'Ask', 'Volume', 'Turnover', 'Updated'])
master = pd.read_csv(master_path, sep='\t')
coverage = pd.read_csv(coverage_path, sep='\t')

print(f"OMXSPI companies: {len(omxspi)}")
print(f"Current instrument_master: {len(master)} companies")
print(f"Current coverage table: {len(coverage)} rows")
omxspi.head(10)

OMXSPI companies: 394
Current instrument_master: 30 companies
Current coverage table: 180 rows


,Fullname,CCY,Last,Change,ChangePct,Bid,Ask,Volume,Turnover,Updated
0,AAK,SEK,258.40,1.2000,+0.46%,258.80,259.20,"376,353","97,145,911",18:00:00
1,ABB Ltd,SEK,709.80,1.6000,+0.22%,710.20,710.40,"485,967","344,276,679",18:00:00
2,AcadeMedia,SEK,94.50,-1.2000,-1.25%,94.20,94.40,"174,418","16,511,663",18:00:00
3,Acast,SEK,32.55,0.4000,+1.24%,32.40,32.90,"249,704","8,092,563",18:00:00
4,Acrinova A,SEK,10.25,0.0500,+0.49%,NaN,NaN,"25,100","257,288",18:00:00
5,Acrinova B,SEK,10.00,-0.1000,-0.99%,10.00,10.15,"16,989","170,774",18:00:00
6,Actic Group,SEK,29.80,-1.9000,-5.99%,29.80,30.10,"75,655","2,293,495",18:00:00
7,Active Biotech,SEK,0.0435,-0.0005,-1.13%,0.0433,0.0435,"7,056,103","310,328",18:00:00
8,AddLife B,SEK,152.10,-1.1000,-0.71%,152.00,152.50,"114,921","17,499,860",18:00:00
9,Addnode Group B,SEK,92.20,-1.3000,-1.39%,91.70,92.50,"106,430","9,740,235",18:00:00


In [24]:
# Find companies in OMXSPI that are not yet in instrument_master
existing_companies = set(master['CompanyName'])
omxspi_companies = set(omxspi['Fullname'])

new_companies = omxspi_companies - existing_companies
print(f"New companies to add: {len(new_companies)}")
print(f"Already in master: {len(omxspi_companies & existing_companies)}")
sorted(new_companies)[:20] if new_companies else "All companies already in master"

New companies to add: 364
Already in master: 30


['AAK',
 'AFRY',
 'AQ Group',
 'AcadeMedia',
 'Acast',
 'Acrinova A',
 'Acrinova B',
 'Actic Group',
 'Active Biotech',
 'AddLife B',
 'Addnode Group B',
 'Alimak Group',
 'Alleima',
 'Alligator Bioscience',
 'Alligo B',
 'Alvotech SDB',
 'Ambea',
 'Annehem Fastigheter B',
 'Anoto Group',
 'Apotea']

In [25]:
# Add new companies to instrument_master with new CIDs
if new_companies:
    # Get the highest existing CID number
    existing_cids = master['CID'].str.extract(r'S(\d+)').astype(int)
    max_cid = existing_cids[0].max() if not existing_cids.empty else 0
    
    new_rows = []
    for i, company in enumerate(sorted(new_companies), start=max_cid + 1):
        new_rows.append({
            'CID': f'S{i}',
            'CompanyName': company,
            'investor_relations_url': ''  # To be filled in step 2
        })
    
    master = pd.concat([master, pd.DataFrame(new_rows)], ignore_index=True)
    master.to_csv(master_path, sep='\t', index=False)
    print(f"Added {len(new_rows)} new companies to instrument_master.csv")
    print(f"Total companies now: {len(master)}")
else:
    print("No new companies to add")

master.head(20)

Added 364 new companies to instrument_master.csv
Total companies now: 394


,CompanyName,ISIN,CID,date refreshed,Nasdaq_url,Active_coverage?,investor_relations_url,CCY,MostRecentStockPrice,TradedStockVolume,StockTurnover,validated
0,ABB Ltd,NaN,S1,NaN,https://www.nasdaq.com/european-market-activit...,True,https://global.abb/group/en/investors/annual-r...,SEK,683.00,453736.0,3.089826e+08,True
1,Addtech B,NaN,S2,NaN,NaN,True,https://www.addtech.com/investors-and-media,SEK,325.60,350644.0,1.141344e+08,True
2,Alfa Laval,NaN,S3,NaN,NaN,True,https://www.alfalaval.com/investors/,SEK,460.60,834570.0,3.841993e+08,True
3,ASSA ABLOY B,NaN,S4,NaN,NaN,True,https://www.assaabloy.com/group/en/investors,SEK,355.80,1924559.0,6.850330e+08,True
4,AstraZeneca,NaN,S5,NaN,NaN,True,https://www.astrazeneca.com/investor-relations...,SEK,1701.00,283942.0,4.806734e+08,True
5,Atlas Copco A,NaN,S6,NaN,NaN,True,https://www.atlascopcogroup.com/en/investors,SEK,164.05,5327339.0,8.735282e+08,True
6,Boliden,NaN,S7,NaN,NaN,True,https://investors.boliden.com/,SEK,498.20,1621368.0,8.060207e+08,True
7,Epiroc A,NaN,S8,NaN,NaN,True,https://www.epirocgroup.com/en/investors/inves...,SEK,206.70,2926875.0,6.055207e+08,True
8,EQT,NaN,S9,NaN,NaN,True,https://eqtgroup.com/shareholders/reports-and-...,SEK,351.80,6554439.0,2.300568e+09,True
9,Ericsson B,NaN,S10,NaN,NaN,True,https://www.ericsson.com/en/investors,SEK,90.42,8433509.0,7.621145e+08,True


In [26]:
# Ensure coverage table has rows for all OMXSPI companies x 6 years
years = list(range(2019, 2025))
required_rows = [(cid, y) for cid in master['CID'] for y in years]
have_rows = set(zip(coverage['Company_Identifier'], coverage['FiscalYear'])) if not coverage.empty else set()
missing_rows = [r for r in required_rows if r not in have_rows]

print(f"Expected rows: {len(required_rows)} ({len(master)} companies × 6 years)")
print(f"Current rows: {len(have_rows)}")
print(f"Missing rows: {len(missing_rows)}")

# Seed missing rows
if missing_rows:
    new_rows = []
    for cid, year in missing_rows:
        company = master.loc[master['CID'] == cid, 'CompanyName'].iloc[0]
        new_rows.append({
            'CompanyName': company,
            'Company_Identifier': cid,
            'What_to_capture': 'Annual report',
            'FiscalYear': year,
            'IR_URL': master.loc[master['CID'] == cid, 'investor_relations_url'].fillna('').iloc[0],
            'Priority': 'Not Downloaded'
        })
    coverage = pd.concat([coverage, pd.DataFrame(new_rows)], ignore_index=True)
    coverage.to_csv(coverage_path, sep='\t', index=False)
    print(f"Added {len(new_rows)} new rows to coverage table")

print(f"Final coverage table shape: {coverage.shape}")
coverage.tail(10)

Expected rows: 2364 (394 companies × 6 years)
Current rows: 180
Missing rows: 2184
Added 2184 new rows to coverage table
Final coverage table shape: (2364, 27)


,CompanyName,Company_Identifier,What_to_capture,FiscalYear,IR_URL,Capture_attempt_date,AgentId,CaptureStatus,CaptureStatusDetails,milestone1_Establish_Connection,...,Probability_Annual_Report,Document_Type,Classification_Reasoning,User_Agent_Used,Validation_Status,Validation_Confidence,Validation_Issues,Validation_Date,Priority,Failure_Reason
2354,mySafety Group B,S393,Annual report,2021,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2355,mySafety Group B,S393,Annual report,2022,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2356,mySafety Group B,S393,Annual report,2023,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2357,mySafety Group B,S393,Annual report,2024,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2358,Öresund,S394,Annual report,2019,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2359,Öresund,S394,Annual report,2020,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2360,Öresund,S394,Annual report,2021,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2361,Öresund,S394,Annual report,2022,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2362,Öresund,S394,Annual report,2023,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
2363,Öresund,S394,Annual report,2024,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Downloaded,NaN
